# Notebook for converting the bounding boxes as per YOLO format

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from ast import literal_eval

In [3]:
train_df = pd.read_csv("train.csv")
train_df.head()

,image_id,width,height,bbox,source
0,b6ab77fd7,1024,1024,"[834.0, 222.0, 56.0, 36.0]",usask_1
1,b6ab77fd7,1024,1024,"[226.0, 548.0, 130.0, 58.0]",usask_1
2,b6ab77fd7,1024,1024,"[377.0, 504.0, 74.0, 160.0]",usask_1
3,b6ab77fd7,1024,1024,"[834.0, 95.0, 109.0, 107.0]",usask_1
4,b6ab77fd7,1024,1024,"[26.0, 144.0, 124.0, 117.0]",usask_1


In [ ]:
def convert(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[1]) / 2.0
    y = (box[2] + box[3]) / 2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return [x, y, w, h]

def convert_to_yolo_label(coco_format_box, w = 1024, h = 1024):
    bbox = literal_eval(coco_format_box)
    xmin = bbox[0]
    xmax = bbox[0] + bbox[2]
    ymin = bbox[1]
    ymax = bbox[1] + bbox[3]
    b = (float(xmin), float(xmax), float(ymin), float(ymax))
    yolo_box = convert((w, h), b)
    if np.max(yolo_box) > 1 or np.min(yolo_box) < 0: # Take this opportunity to check that conversion works
        print("BOX HAS AN ISSUE")
    return yolo_box

In [ ]:
train_df['yolo_box'] = train_df.bbox.apply(convert_to_yolo_label)

In [ ]:
unique_img_ids = train_df.image_id.unique()

In [ ]:
len(unique_img_ids)

In [ ]:
from tqdm import tqdm

In [ ]:
for img_id in tqdm(unique_img_ids):
    filt_df = train_df.query("image_id == @img_id")
    all_boxes = filt_df.yolo_box.values
    file_name = img_id+".txt"

    s = "0 %s %s %s %s \n"
    with open(file_name, 'a') as file:
        for i in all_boxes:
            new_line = (s % tuple(i))
            file.write(new_line)  

In [ ]:
import glob
all_imgs = glob.glob("../input/global-wheat-detection/train/*.jpg")
all_imgs = [i.split("/")[-1].replace(".jpg", "") for i in all_imgs]

In [ ]:
negative_images = set(all_imgs) - set(unique_img_ids)

In [ ]:
for i in tqdm(list(negative_images)):
    file_name = i+".txt"
    with open(file_name, 'w') as fp: 
        pass